In [1]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import Perceptron

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical

from google.colab import drive

from matplotlib import pyplot as plt
from sklearn.decomposition import PCA

from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier

def print_tree(clf, X_test):
  node_indicator = clf.decision_path(X_test)
  leaf_id = clf.apply(X_test)
  feature = clf.tree_.feature
  threshold = clf.tree_.threshold

  sample_id = 0
  # obtain ids of the nodes `sample_id` goes through, i.e., row `sample_id`
  node_index = node_indicator.indices[
      node_indicator.indptr[sample_id] : node_indicator.indptr[sample_id + 1]
  ]

  print("Rules used to predict sample {id}:\n".format(id=sample_id))
  for node_id in node_index:
      # continue to the next node if it is a leaf node
      if leaf_id[sample_id] == node_id:
          continue

      # check if value of the split feature for sample 0 is below threshold
      if X_test[sample_id, feature[node_id]] <= threshold[node_id]:
          threshold_sign = "<="
      else:
          threshold_sign = ">"

      print(
          "decision node {node} : (X_test[{sample}, {feature}] = {value}) "
          "{inequality} {threshold})".format(
              node=node_id,
              sample=sample_id,
              feature=feature[node_id],
              value=X_test[sample_id, feature[node_id]],
              inequality=threshold_sign,
              threshold=threshold[node_id],
          )
      )

import os
    
# File input
def read_in_data():
    print(os.getcwd())
#     path = '/content/drive/My Drive/Colab Notebooks/cosmic_sleuth/data/3FHL_50.txt'
    # path = '/content/drive/My Drive/Colab Notebooks/cosmic_sleuth/data/four_sources_plus_background.txt'
    # path = '/content/drive/My Drive/Colab Notebooks/cosmic_sleuth/data/sources_ft_numberlabels.txt'

#     drive.mount('/content/drive')
    with open(path, 'r') as f:
      temp = np.genfromtxt(f,delimiter=' ', dtype= None)

    #label_size = 3
    # label_size = 1
    
    X = np.array([[a, b, c, d, e, f] for (a, b, c, d, e, f, g) in temp])
    y = np.array([g for (a, b, c, d, e, f, g) in temp])
    # X = temp[:, :-1]
    # y = temp[:, -1]

    return X, y

# Find all unique x,y,x coordinates, assign them a label
def enumerate_unique_labels(y):
    y_options = np.unique(y, axis=0)
    # Convert x,y,z coordinate into an enumerated label
    print(y_options)
    print(np.linalg.norm(y_options[0]))
    print(np.linalg.norm(y_options[1]))
    print(np.linalg.norm(y_options[2]))
    print(np.linalg.norm(y_options[3]))
    y_labels = np.array([np.where(y_options == yi)[0][0] for yi in y])
    return y_labels

def eval_classifier(clf, X_train, X_test, y_train, y_test):
    clf = clf.fit(X_train, y_train)

    print('\t', clf.score(X_train, y_train), end="\t")
    print(clf.score(X_test, y_test))

    return clf

def mlp():
    # Create the model
    model = Sequential()
    model.add(Dense(10, input_dim = 6, activation='relu'))
    model.add(Dense(25, activation='relu'))
    model.add(Dense(50, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

def eval_mlp(model, X_train, X_test, y_train, y_test):
    # Convert target classes to categorical ones
    Y_train = to_categorical(y_train)
    Y_test = to_categorical(y_test)

    model.fit(X_train, Y_train, epochs=200, batch_size=45, verbose=1)
    # Test the model after training
    test_results = model.evaluate(X_test, Y_test, verbose=1)
    print(f'Mulilayer Perceptron results - Loss: {test_results[0]} - Accuracy: {test_results[1]}%')
    
def eval_Linregression(X_train, X_test, y_train, y_test):
    possibleys = [0, 1, 2, 3]
    clf = LinearRegression().fit(X_train, y_train)
    yhat = clf.predict(X_test)
    yclass = yhat
    for i in range(yhat.shape[0]): 
      yclass[i] = np.argmin(np.linalg.norm(possibleys - yhat[i]))
    score = sum([y1 == y2 for (y1, y2) in zip(yclass, y_test)])
    score = 1.0 * score / y_test.shape[0]

    print('\tLinearRegression Score\t ', score)
    return clf

def main():
    X, y = read_in_data()
    # y_labels = enumerate_unique_labels(y)
    # print(np.unique(y_labels))
    y_options = np.unique(y, axis=0)
    y_labels = np.array([np.where(y_options == yi)[0][0] for yi in y])
    print(y_labels)

    U, S, Vh = np.linalg.svd(X - X.mean(axis=0))

    X_train, X_test, y_train, y_test = train_test_split(U[:, :6], y_labels, test_size=0.2)

    # Evaluate many different classifiers
    print('Decision Tree:  ', end='\t')
    clf = eval_classifier(DecisionTreeClassifier(), X_train, X_test, y_train, y_test)
    plt.figure(figsize=(20,20))  # set plot size (denoted in inches)
    # tree.plot_tree(clf, fontsize=10)
    # plt.show()

    print('Random Forest:', end='\t')
    eval_classifier(RandomForestClassifier(n_estimators=1000), X_train, X_test, y_train, y_test)

    print('LightGBM:', end='\t')
    eval_classifier(LGBMClassifier(objective='multiclass'), X_train, X_test, y_train, y_test)

    print('1-NN Classifier:', end='\t')
    eval_classifier(KNeighborsClassifier(n_neighbors=1), X_train, X_test, y_train, y_test)
    print('3-NN Classifier:', end='\t')
    eval_classifier(KNeighborsClassifier(n_neighbors=3), X_train, X_test, y_train, y_test)
    print('10-NN Classifier:', end='\t') 
    eval_classifier(KNeighborsClassifier(n_neighbors=10), X_train, X_test, y_train, y_test)
    print('GaussianNB Class.:', end='\t')
    eval_classifier(GaussianNB(), X_train, X_test, y_train, y_test)
    # print('LinearRegression:', end='\t')
    # eval_classifier(LinearRegression(), X_train, X_test, y_train, y_test)
    # print('LinearRegression:', end='\t')
    # eval_Linregression(X_train, X_test, y_train, y_test)
    # print('LogisticRegression:', end='\t')
    # eval_classifier(LogisticRegression(), X_train, X_test, y_train, y_test)
    # print('Perceptron Class.:', end='\t')
    # eval_classifier(Perceptron(), X_train, X_test, y_train, y_test)

    # model = mlp()
    # print(model.summary())
    # print(X_train.shape)

    # eval_mlp(model, X_train, X_test, y_train, y_test)

main()


ModuleNotFoundError: No module named 'tensorflow'